In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants
from tqdm import tqdm  # for progress bar (optional)
import time
from scipy import special as sp
from scipy.interpolate import interp1d

In [ ]:
# def lj_potential(r):
#     sigma = 3.405   # Angstrom (Ar) given in the exercise
#     epsi = 0.997
#     return 4 * epsi * ((sigma / r) ** 12 - (sigma / r) ** 6)
# Lennard-Jones potential
epsi = 1
sigma = 1
lj_potential = lambda r: 4 * epsi * ((sigma / r) ** 12 - (sigma / r) ** 6)

""" # Function to calculate distance between two points with periodic boundary conditions
def distance(r1, r2):
    dx = r1[0] - r2[0]
    dy = r1[1] - r2[1]
    dx -= np.round(dx / L) * L
    dy -= np.round(dy / L) * L
    return np.sqrt(dx**2 + dy**2) """

distance = lambda r1, r2, L: np.sqrt(((r1[0] - r2[0] - np.round((r1[0] - r2[0]) / L) * L)**2) + ((r1[1] - r2[1] - np.round((r1[1] - r2[1]) / L) * L)**2))

""" def TotEnergy(positions, L, N, T):
    E = scipy.constants.Boltzmann * T
    for i in range(N):
        for j in range(i+1, N):
            E += lj_potential(distance(positions[i], positions[j], L))
    return E """

# Compute radial distribution function g(r)
def calculate_radial_distribution(positions, dr, rho, L):
    N = len(positions)
    max_bin_index = int(L / 2 / dr)
    g_values = np.zeros(max_bin_index)

    for i in range(N):
        for j in range(i+1, N):
            r = distance(positions[i], positions[j], L)
            if r < L / 2:
                bin_index = min(int(r / dr), max_bin_index - 1)
                g_values[bin_index] += 2  # Count each pair only once

    # Normalize g(r)
    for i in range(len(g_values)):
        r_lower = i * dr
        r_upper = (i + 1) * dr
        shell_volume = np.pi * ((r_upper)**2 - (r_lower)**2)
        g_values[i] /= shell_volume * rho * N

    r_values = np.arange(dr, (max_bin_index + 1) * dr, dr)
    return r_values, g_values

def calculate_radial_distribution2(positions, dr, rho, L):
    hist, bins = np.histogramdd(positions, bins=np.linspace(0, L, 50))
    r = bins[:-1, 0] + bins[1:, 0]  # Bin centers
    dr = bins[1, 0] - bins[0, 0]
    n_atoms = len(positions)
    volume = L**2
    rdf = hist.flatten() / (n_atoms * dr * volume)
    return r, rdf

In [ ]:
## PARAMETERS ##
N = 242  # Number of atoms
T = 0.5 # Temperature
p = 0.96  # Density
L = np.sqrt(N / p)  # Box size
D = 0.15  # Displacement amplitude
n_steps = 10000000

In [ ]:
# Initialize positions randomly
positions = np.random.rand(N, 2) * L

# Initialize energy
energy = 0.0

# Perform Monte Carlo steps
n_accept = np.zeros(n_steps)

with tqdm(total=n_steps) as pbar:
    for step in range(n_steps):
        # Randomly select an atom
        atom_index = np.random.randint(N)
        
        # Calculate energy before displacement
        old_energy = scipy.constants.Boltzmann * T
        for i in range(N):
            if i != atom_index:
                old_energy += lj_potential(distance(positions[atom_index], positions[i], L))
        
        # Propose a random displacement
        displacement = (np.random.rand(2) - 0.5) * D
        new_position = positions[atom_index] + displacement
        if np.any(new_position < 0) or np.any(new_position >= L):
            continue
        
        # Apply periodic boundary conditions
        new_position = np.mod(new_position, L)
        
        # Calculate energy after displacement
        new_energy = scipy.constants.Boltzmann * T
        for i in range(N):
            if i != atom_index:
                new_energy += lj_potential(distance(new_position, positions[i], L))
        
        # Metropolis criterion
        delta_energy = new_energy - old_energy
        if delta_energy < 0 or np.random.rand() < np.exp(-delta_energy / T):
            positions[atom_index] = new_position
            energy += delta_energy
            n_accept[step] = 1

        pbar.update()

In [ ]:
r_values, g_values = calculate_radial_distribution(positions, 0.1, rho = p, L = L)

""" # Plot the acceptance rate
plt.figure()
plt.plot(np.cumsum(n_accept) / np.arange(1, n_steps + 1))
plt.xlabel('Step')
plt.ylabel('Acceptance rate')
plt.title('Acceptance rate') """

# Calculate the average energy per atom
U_values = np.zeros(N*2)
for i in range(N):
    for j in range(i+1, N):
        r = distance(positions[i], positions[j], L)
        U_values[i] += lj_potential(r)
        U_values[j] += lj_potential(r)

E_avg = np.mean(U_values)
E_std = np.std(U_values)

print("Average energy:", E_avg)
print("Energy standard deviation:", E_std)

# Plot g(r)
plt.figure()
plt.plot(r_values[:-1], g_values[:-1])
plt.xlabel('r')
plt.ylabel('g(r)')
plt.title('Radial Distribution Function')

# Plot the atom positions
plt.figure()
plt.scatter(positions[:, 0], positions[:, 1])
plt.xlabel('x')
plt.ylabel('y')
plt.title('Final Atom Positions')
plt.grid()

# Show plots
plt.show()

In [ ]:
from rdfpy import rdf

r, g_r = rdf(positions, 0.08, p, parallel=False)

plt.figure()
plt.plot(g_r, r)
plt.xlabel('r')
plt.ylabel('g(r)')
plt.title('Radial Distribution Function')
plt.show()